In [1]:
import textacy
import spacy
from spacy.tokens import Span
from spacy.matcher import Matcher
from ipywidgets import HTML
from spacy import displacy

In [2]:
de_text = "Bei der Auslegung einer Willenserklärung ist der wirkliche Wille zu erforschen und nicht an dem buchstäblichen Sinn der Worte ist zu haften, d.h. ohne daran zu haften."

blank_de = textacy.make_spacy_doc(de_text, lang='de_core_news_lg')
de_doc = blank_de

In [3]:
# import Matcher and rewrite the previous code using it, to print the same results
from spacy.matcher import Matcher
from ipywidgets import HTML

matcher = Matcher(de_doc.vocab)

def add_span_attributes(matcher, doc, i, matches, span_attrs=None):
    match_id, start, end = matches[i]
    label_ = doc.vocab.strings[match_id]
    span = Span(doc, start, end, label=label_)
    # span.set_extension(label_, getter=lambda x: True if span.label == label_ else False,
    #                    force=True)
    doc.ents = textacy.extract.entities(doc)
    print(doc.ents)
    if span_attrs:
        for attr, val in span_attrs.items():
            setattr(span._, attr, val)

# def clean_matches_on_match(matcher, doc, i, matches):
#     unique_matches = []
#     for match_text, start, end in matches:
#         if start not in [start for _, start, end in unique_matches]:
#             unique_matches.append((match_text, start, end))
#     match_id, start, end = unique_matches[i]
#     span = Span(doc, start, end, label=match_id, span_id=match_id)
#     doc.ents = list(doc.ents) + [span] if span not in doc.ents else [doc.ents]

matcher.add('ohne_zu__inf', [[{'ORTH': 'ohne'}, 
                              {'IS_ALPHA': True, "OP": "*"}, 
                              {'ORTH': 'zu'}, 
                              {'POS': 'VERB', 'OP': '+'}]], 
            # on_match=add_span_attributes
            # on_match=Span.set_extension('ohne_zu__inf', force=True, getter=lambda span: True if 'ohne' in span.text else False)
            )

matcher.add('zu__inf__sein', [[{"POS": "AUX", "LEMMA": {"IN": ["sein"]}}, 
                               {'IS_ALPHA': True, "OP": "*"}, 
                               {'LEMMA': 'zu'}, 
                               {'POS': 'VERB', 'OP': '+'}]], 
            # on_match=add_span_attributes
            # on_match=Span.set_extension('zu__inf__sein', force=True, getter=lambda span: True if 'ist' in span.text else False)
            )

matcher.pipe(de_doc, as_tuples=True)
matches = matcher(de_doc)


de_doc.spans['custom'] = [
    # Span(de_doc, 2, 5, 'ΝΟΜ'),
    # Span(de_doc, 5, 6, 'ΓΛΩ'),
    # Span(de_doc, 8, 9, 'ΝΟΜ'),
    # Span(de_doc, 9, 11, 'ΓΛΩ'),
    # Span(de_doc, 12, 18, 'ΠΣΔ'),
    # Span(de_doc, 17, 18, 'ΓΛΩ'),
]

# make sure the next iteration doesn't match the same tokens again
unique_matches = []
for match_text, start, end in matches:
    if start not in [start for _, start, end in unique_matches]:
        unique_matches.append((match_text, start, end))

for match_id, start, end in unique_matches:
    string_id = de_doc.vocab.strings[match_id]
    span = de_doc[start:end]
    print()
    display(HTML(f'<strong>{span.text}</strong> (<code>{string_id}</code>)'))
    # append the span to the list of spans
    de_doc.spans['custom'].append(span)

# print("Noun phrases:", '\n'.join([chunk.text for chunk in de_doc.noun_chunks]), sep='\n' + '-'*25 +'\n', end='\n\n')
# print(de_doc.spans['custom'])

print('ohne zu _inf_:', [de_doc[start:end].text for _, start, end in matches if de_doc[start:end]._.ohne_zu__inf])
print('zu _inf_ sein:', [de_doc[start:end].text for _, start, end in matches if de_doc[start:end]._.zu__inf__sein])

# print('ohne zu _inf_:', list(set([span.text for span in de_doc.spans['custom'] if span._.ohne_zu__inf])))
# print('zu _inf_ sein:', list(set([span.text for span in de_doc.spans['custom'] if span._.zu__inf__sein])))


# custom spans
de_doc.spans['custom'] = [
    Span(de_doc, 2, 5, 'ΝΟΜ'),
    Span(de_doc, 5, 6, 'ΓΛΩ'),
    Span(de_doc, 8, 9, 'ΝΟΜ'),
    Span(de_doc, 9, 11, 'ΓΛΩ'),
    Span(de_doc, 12, 18, 'ΠΣΔ'),
    Span(de_doc, 17, 18, 'ΓΛΩ'),
] + list(de_doc.spans['custom'])

options = {
    'spans_key': 'custom', 
    'colors': {
        'ΝΟΜ': '#79b8e1', 
        'ΓΛΩ': '#ffa147', 
        'ΠΣΔ': '#9f8df0'
    },
}

displacy.render(de_doc, style='span', options=options)

HTML(value='<strong>ist der wirkliche Wille zu erforschen</strong> (<code>zu__inf__sein</code>)')

HTML(value='<strong>ist zu haften</strong> (<code>zu__inf__sein</code>)')

HTML(value='<strong>ohne daran zu haften</strong> (<code>ohne_zu__inf</code>)')

AttributeError: [E046] Can't retrieve unregistered extension attribute 'ohne_zu__inf'. Did you forget to call the `set_extension` method?